In [93]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization,ZeroPadding2D
import cv2
import numpy as np
import os
from tqdm import tqdm
import tensorflow as tf

In [94]:
import zipfile
local_zip='/content/Student Projects.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [95]:
celosia='/content/CELOSIA ARGENTEA L'
crowfoot='/content/CROWFOOT GRASS'
chloris='/content/PURPLE CHLORIS'


In [96]:
IMG_SZ=227
labels=[]

In [97]:
def image_read(TRAIN_DATA):
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DATA)):
        path = os.path.join(TRAIN_DATA,img)
        img = cv2.imread(path)
        if img is not None:
          img = cv2.resize(img, (IMG_SZ,IMG_SZ))
          img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
          training_data.append(np.array(img))
    return training_data

In [98]:
def labels_funct(data,class_value):
  y=np.ones(len(data),dtype=int)*class_value
  return y

In [99]:
celosia_data=image_read(celosia)


100%|██████████| 60/60 [00:00<00:00, 200.80it/s]


In [100]:
crowfoot_data=image_read(crowfoot)


100%|██████████| 60/60 [00:00<00:00, 356.25it/s]


In [101]:
chloris_data=image_read(chloris)

100%|██████████| 73/73 [00:00<00:00, 206.18it/s]


In [102]:
crowfoot_label=  labels_funct(crowfoot_data,0)

In [103]:
celosia_label=  labels_funct(celosia_data,1)

In [104]:
chloris_label=  labels_funct(chloris_data,2)

In [105]:
chloris_label.shape

(73,)

In [106]:
celosia_label.shape

(60,)

In [107]:
crowfoot_label.shape

(60,)

In [108]:
dataset=np.concatenate((crowfoot_data,celosia_data,chloris_data))

In [109]:
dataset.shape

(193, 227, 227, 3)

In [110]:
class_labels=np.concatenate((crowfoot_label,celosia_label,chloris_label))

In [111]:
class_labels.shape

(193,)

In [112]:
import sklearn.model_selection as ms

In [113]:
x_train,x_test,y_train,y_test=ms.train_test_split(dataset,class_labels,test_size=0.3,random_state=42)

In [114]:
x_train.shape,x_test.shape

((135, 227, 227, 3), (58, 227, 227, 3))

In [115]:
from tensorflow.keras.utils import to_categorical
y_train_onehot= to_categorical(y_train)
y_test_onehot=to_categorical(y_test)

In [127]:
alexnet=Sequential()
#Layer 1
alexnet.add(Conv2D(96,(11,11), input_shape=(227,227,3),padding='same'))
alexnet.add(BatchNormalization())
conv1=Activation('relu')
alexnet.add(conv1)
max_pool_1=MaxPooling2D(pool_size=(2,2))
alexnet.add(max_pool_1)

#Layer 2
alexnet.add(Conv2D(256, (5, 5), padding='same'))
alexnet.add(BatchNormalization())
conv2=Activation('relu')
alexnet.add(conv2)
max_pool_2=MaxPooling2D(pool_size=(2, 2))
alexnet.add(max_pool_2)

# Layer 3
alexnet.add(ZeroPadding2D((1, 1)))
alexnet.add(Conv2D(512, (3, 3), padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 4
alexnet.add(ZeroPadding2D((1, 1)))
alexnet.add(Conv2D(1024, (3, 3), padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))

# Layer 5
alexnet.add(ZeroPadding2D((1, 1)))
alexnet.add(Conv2D(1024, (3, 3), padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
max_pool_5=MaxPooling2D(pool_size=(2, 2))
alexnet.add(max_pool_5)

# Layer 6
alexnet.add(Flatten())
alexnet.add(Dense(512))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(Dropout(0.5))

# Layer 7
alexnet.add(Dense(512))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(Dropout(0.5))

# Layer 8
alexnet.add(Dense(3))
alexnet.add(BatchNormalization())
alexnet.add(Activation('softmax'))

In [128]:
alexnet.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_33 (Conv2D)          (None, 227, 227, 96)      34944     
                                                                 
 batch_normalization_40 (Bat  (None, 227, 227, 96)     384       
 chNormalization)                                                
                                                                 
 activation_40 (Activation)  (None, 227, 227, 96)      0         
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 113, 113, 96)     0         
 g2D)                                                            
                                                                 
 conv2d_34 (Conv2D)          (None, 113, 113, 256)     614656    
                                                                 
 batch_normalization_41 (Bat  (None, 113, 113, 256)   

In [129]:
alexnet.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

In [130]:
history=alexnet.fit(x_train[0:227], y_train_onehot[0:227], batch_size=8, epochs=20, validation_split=0.1, shuffle=True)

Epoch 1/20
16/16 [==============================] - 353s 22s/step - loss: 1.2678 - accuracy: 0.3471 - val_loss: 561.4216 - val_accuracy: 0.6429
Epoch 2/20
16/16 [==============================] - 336s 21s/step - loss: 1.0359 - accuracy: 0.4876 - val_loss: 324.4914 - val_accuracy: 0.6429
Epoch 3/20
16/16 [==============================] - 337s 21s/step - loss: 0.8393 - accuracy: 0.6364 - val_loss: 45.0792 - val_accuracy: 0.6429
Epoch 4/20
16/16 [==============================] - 336s 21s/step - loss: 0.7937 - accuracy: 0.6529 - val_loss: 42.8173 - val_accuracy: 0.6429
Epoch 5/20
16/16 [==============================] - 338s 21s/step - loss: 0.6759 - accuracy: 0.7273 - val_loss: 16.8664 - val_accuracy: 0.2857
Epoch 6/20
16/16 [==============================] - 338s 21s/step - loss: 0.7373 - accuracy: 0.7190 - val_loss: 12.3893 - val_accuracy: 0.5714
Epoch 7/20
16/16 [==============================] - 340s 21s/step - loss: 0.6745 - accuracy: 0.7603 - val_loss: 7.6707 - val_accuracy: 0.214

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=8, epochs=20, verbose=1, validation_split=0.1)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {test_loss:.3f}, Test accuracy: {test_acc:.3f}')

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/20
6750/6750 [==============================] - 69s 10ms/step - loss: 0.1076 - accuracy: 0.9665 - val_loss: 0.0443 - val_accuracy: 0.9883
Epoch 2/20
5282/6750 [======================>.......] - ETA: 13s - loss: 0.0405 - accuracy: 0.9873

In [ ]:
alexnet.save('Weed_Detection _Model.h5')